# Gross Box Office Data

This notebook explores grabbing the data we need to establish the lifetime gross revenues for films. I downloaded a scraper built for Box Office Mojo, but honestly, it looksl like we can get what we need from five pages that have URLs with offsets:
```
https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=200
```
With that noted, let's load up `requests`, grab some pages, parse them with `BeautifulSoup`, and then explore what data structure will let us pair up revenues with film titles we have from file names.

In [26]:
import urllib.request
from bs4 import BeautifulSoup
from io import StringIO # BeautifulSoup now prefers this as a wrapper
import pandas as pd

In [10]:
# URL from above minus offset
base_url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset="

# Offsets to add to URL
offsets = [0, 200, 400, 600, 800]

In [13]:
myurl = f"{base_url}str[0]"
myconnection = urllib.request.urlopen(myurl)
myhtml = myconnection.read()

In [14]:
# Check to see that things work
print(myhtml[0:100])

b'<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo"><head><script>var aPageStart = (new Date'


In [18]:
with open('../data/bom.html', 'wb') as f:
    f.write(myhtml)

It looks like everything is packed into a table with rows that look like this:

```html
<tr>
    <td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">1</td>
    <td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt0499549/?ref_=bo_cso_table_1">Avatar</a></td>
    <td class="a-text-right mojo-field-type-money">$2,923,706,026</td>
    <td class="a-text-right mojo-field-type-money">$785,221,649</td>
    <td class="a-text-right mojo-field-type-percent">26.9%</td>
    <td class="a-text-right mojo-field-type-money">$2,138,484,377</td><td class="a-text-right mojo-field-type-percent">73.1%</td>
    <td class="a-text-left mojo-field-type-year"><a class="a-link-normal" href="/year/2009/?ref_=bo_cso_table_1">2009</a></td>
</tr>
```
This table is available in the bom-table.html file in the data directory. You'll see that I've deleted everything from the file that wasn't the table, and then I have isolated the single row above so I can begin to determine how to "parse" this using BeautifulSoup. My goal is to write this directly to a pandas dataframe, if I can.


In [37]:
# The BeautifulSoup incantation
soup = BeautifulSoup(myhtml,'lxml')

# This is easy
table = soup.find_all('table')

In [ ]:
# Send it to a dataframe
df = pd.read_html(StringIO(str(table)))[0]

In [34]:
# Did we get all 200 rows?
df.shape

(200, 8)

In [30]:
# How's it look?
df.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1,Avatar,"$2,923,706,026","$785,221,649",26.9%,"$2,138,484,377",73.1%,2009
1,2,Avengers: Endgame,"$2,799,439,100","$858,373,000",30.7%,"$1,941,066,100",69.3%,2019
2,3,Avatar: The Way of Water,"$2,320,250,281","$684,075,767",29.5%,"$1,636,174,514",70.5%,2022
3,4,Titanic,"$2,264,750,694","$674,292,608",29.8%,"$1,590,458,086",70.2%,1997
4,5,Star Wars: Episode VII - The Force Awakens,"$2,071,310,218","$936,662,225",45.2%,"$1,134,647,993",54.8%,2015


Listen, we could write a function, or a `for` loop, but honestly we only have to do this four more times, and I think it'll just be easier to put everything into a cell and then run that cell four times with an edit to the offset number and to the dataframe. We'll then merge all the dataframes at the end and save that to a CSV. 

In [52]:
myurl = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=1200"
myconnection = urllib.request.urlopen(myurl)
myhtml = myconnection.read()
soup = BeautifulSoup(myhtml,'lxml')
table = soup.find_all('table')
df_1200 = pd.read_html(StringIO(str(table)))[0]
df_1200.head()

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1201,P.S. I Love You,"$156,846,321","$53,706,790",34.2%,"$103,139,531",65.8%,2007
1,1202,The Monuments Men,"$156,706,638","$78,031,620",49.8%,"$78,675,018",50.2%,2014
2,1203,Mystic River,"$156,595,191","$90,135,191",57.6%,"$66,460,000",42.4%,2003
3,1204,Under Siege,"$156,563,139","$83,563,139",53.4%,"$73,000,000",46.6%,1992
4,1205,This Means War,"$156,491,279","$54,760,791",35%,"$101,730,488",65%,2012


So now I have dataframe with offsets of 0, 200, 400, 600, 800, 1000, and 1200. That's 7 dataframes with 1400 box office results! Let's get them all into a single dataframe and then save the 1400 lines to a CSV.

In [55]:
df_all = pd.concat([df, df_200, df_400, df_600, df_800, df_1000, df_1200], ignore_index=True)

In [56]:
df_all.shape

(1400, 8)

In [57]:
df_all

,Rank,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,1,Avatar,"$2,923,706,026","$785,221,649",26.9%,"$2,138,484,377",73.1%,2009
1,2,Avengers: Endgame,"$2,799,439,100","$858,373,000",30.7%,"$1,941,066,100",69.3%,2019
2,3,Avatar: The Way of Water,"$2,320,250,281","$684,075,767",29.5%,"$1,636,174,514",70.5%,2022
3,4,Titanic,"$2,264,750,694","$674,292,608",29.8%,"$1,590,458,086",70.2%,1997
4,5,Star Wars: Episode VII - The Force Awakens,"$2,071,310,218","$936,662,225",45.2%,"$1,134,647,993",54.8%,2015
...,...,...,...,...,...,...,...,...
1395,1396,A League of Their Own,"$132,440,069","$107,533,928",81.2%,"$24,906,141",18.8%,1992
1396,1397,Be Somebody,"$132,326,332",-,-,"$132,326,332",100%,2021
1397,1398,The Three Musketeers,"$132,274,484","$20,374,484",15.4%,"$111,900,000",84.6%,2011
1398,1399,Suzume,"$132,181,655","$10,932,037",8.3%,"$121,249,618",91.7%,2022


In [58]:
df_all.to_csv("../data/boxoffice.csv")